In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import PlaceCellAnalysis as pc
os.sys.path.append('../')
import utilities as u
import preprocessing as pp
import behavior as b
import BayesianDecoding as bd
import SimilarityMatrixAnalysis as sm
import pickle


%load_ext autoreload
%autoreload 2

%matplotlib inline

C:\Users\mplitt\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pp.load_session_db()
df = df[df['RewardCount']>20]
df = df[df['Imaging']==1]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])

mouse = '4139224.3'

df_mouse = df[df['MouseName'].str.match(mouse)]
df_sess = df_mouse[df_mouse['Track'].str.match('TwoTower_Timeout')]
df_sess = df_sess[df_sess['ImagingRegion'].str.match('CA1') | df_sess['ImagingRegion'].str.match('')]

In [ ]:
mice = ['4139190.1','4139212.2','4139219.2','4139219.3','4139224.2','4139224.3','4139224.5']


In [ ]:

UUU,SSS = np.zeros([5,5]),np.zeros([450,450])
for mouse in mice:
# mouse = '4139224.3'

    df_mouse = df[df['MouseName'].str.match(mouse)]
    df_sess = df_mouse[df_mouse['Track'].str.match('TwoTower_Timeout')]
    df_sess = df_sess[df_sess['ImagingRegion'].str.match('CA1') | df_sess['ImagingRegion'].str.match('')]

    dirbase = "G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\%s\\" % mouse

    U0,U1 = [],[]
    UU = np.zeros([5,5])
    SS= np.zeros([450,450])
    count = 0
    for i in range(df_sess.shape[0]):
        try:
            fname = "%s\\%s_%s_%d_" % (dirbase,mouse,df_sess['DateFolder'].iloc[i],df_sess['SessionNumber'].iloc[i])
            with open(fname+"results_corr.pkl",'rb') as f:
                sm_res = pickle.load(f)

            S_corr = sm_res['S']

        #     VRDat=pp.behavior_dataframe(df_sess['data file'].iloc[i])
            #VRDat,C,Cd, Spikes, A = pp.load_scan_sess(df_sess.iloc[i])
            #trial_info, tstart_inds, teleport_inds = u.by_trial_info(VRDat)
            if S_corr.shape[0]>200:
                ms = np.array([0, .25, .5, .75,1.])

            else:
                ms = np.array([0.,1.])
        #     ms = np.unique(trial_info['morphs'])
            m = ms.shape[0]

            U = np.zeros([m,m,S_corr.shape[2]])
            for p in range(S_corr.shape[2]):
                U[:,:,p], trash = sm.morph_mean_simmat(S_corr[:,:,p],m)
            f_U,ax_U = plt.subplots(figsize=[5,5])
            ax_U.imshow(np.nanmean(U,axis=-1),cmap='Greys')
            f_U.savefig(fname+"U_corr.pdf",format='pdf')
            f_U.savefig(fname+"U_corr.svg",format='svg')
            
            S_corr[np.isnan(S_corr)]=0
            U[np.isnan(U)]=0
            if m ==5:
                SS += np.nanmean(S_corr,axis=-1)
                UU += np.nanmean(U,axis=-1)
                count += 1
            f,ax = plt.subplots(figsize=[5,3])


            ax.errorbar(ms,np.nanmean(U[0,:,:],axis=-1),np.nanstd(U[0,:,:],axis=-1),
                               color=plt.cm.cool(np.float(0)),ecolor=plt.cm.cool(np.float(0)))
            U0.append(np.nanmean(U[0,:,:],axis=-1))

            ax.errorbar(ms,np.nanmean(U[-1,:,:],axis=-1),np.nanstd(U[-1,:,:],axis=-1),
                               color=plt.cm.cool(np.float(1)),ecolor=plt.cm.cool(np.float(1)))
            U1.append(np.nanmean(U[-1,:,:],axis=-1))

            f.savefig(fname+"Uline_corr.pdf",format='pdf')
            f.savefig(fname+"Uline_corr.svg",format='svg')

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.set_ylim([.1, .3])
            ax.set_yticks([.1,.2,.3])
            ax.set_xticks([0,.25,.5,.75,1.])
        except:
            pass


    SS/=count
    UU/=count
    
    f_S,ax_S = sm.plot_simmat(SS,5)
    f_S.savefig("G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\%s_S_corr.pdf" % mouse,format='pdf')
    f_S.savefig( "G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\%s_S_corr.svg" % mouse,format='svg')

    f_U,ax_U=plt.subplots()
    ax_U.imshow(UU,cmap='Greys')
    f_U.savefig("G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\%s_U_corr.pdf" % mouse,format='pdf')
    f_U.savefig( "G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\%s_U_corr.svg" % mouse,format='svg')
    
    SSS+=SS
    UUU+=UU

C:\Users\mplitt\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in true_divide
C:\Users\mplitt\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in true_divide
C:\Users\mplitt\Anaconda3\lib\site-packages\matplotlib\colors.py:897: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
C:\Users\mplitt\Anaconda3\lib\site-packages\numpy\ma\core.py:716: UserWarning: Warning: converting a masked element to nan.
  data = np.array(a, copy=False, subok=subok)
C:\Users\mplitt\Anaconda3\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
f_S,ax_S = sm.plot_simmat(SSS,5)
f_S.savefig("G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\MEAN_S_corr.pdf" ,format='pdf')
f_S.savefig( "G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\MEAN_S_corr.svg",format='svg')

f_U,ax_U=plt.subplots()
ax_U.imshow(UUU,cmap='Greys')
f_U.savefig("G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\MEAN_U_corr.pdf" ,format='pdf')
f_U.savefig( "G:\\My Drive\\Figures\\TwoTower\\COSYNE2019\\SimMats\\MEAN_U_corr.svg",format='svg')